# Bootstrap Filter

In [2]:
import numpy as np
import math
import scipy.stats
import seaborn
import matplotlib.pyplot as plt
import copy
%matplotlib inline 

In [43]:
class BootstrapFilter(object):
    
    def __init__(self, prior, tm, N=5000, noise = 0.1):
        #what parameters?
        # prior
        #transition matrix
        # number of particles
        self.tm = np.cumsum(tm, axis=1)
        
        #init particles
        #particles as a np array
        #each particle has: state identifier, Mean, Var
        state = prior[0]
        n = N*state[-1]
        name = np.full((n), 0)
        mean = np.full((n), state[0])\
         + np.random.normal(scale=noise, size=n)
        var = np.full((n), state[1])\
         + np.random.normal(scale=noise, size=n)
        self.particles = np.stack((name,mean,var), axis=-1)
        
        for i, state in enumerate(prior[1:]):
            n = N*state[-1]
            name = np.full((n), i+1)
            mean = np.full((n), state[0])\
             + np.random.normal(scale=noise, size=n)
            var = np.full((n), state[1])\
             + np.random.normal(scale=noise, size=n)
            p = np.stack((name,mean,var), axis=-1)
            self.particles = np.concatenate((self.particles, p))
        print self.particles.shape
        
        
    def importantSampling(self, evidence):
        
        for p in self.particles:
            #markov transition
            pAfterTransition = self.markovTransition(p)
            #weightParticle
            unWeightp = self.weightParticles(p, evidence)
        
        #normalise weights
        self.weights = normWeights(unWeights)
        
    def resample(self):
        #dont know yet
        pass
        
    def markovTransition(self, p):
        #markov transition
        r = np.random.rand()
        newState = np.searchsorted(self.tm[p[0]], r)
        p[0] = newState
        #TODO update mean and var of particle to new state
        pass
    
    def weightParticle(self, evidence):
        # compute likelihood
        pass

In [64]:
tm = [[0.8,0.2],[0.1,0.9]]
print tm
a = np.cumsum(tm, axis=1)
print a


[[0.8, 0.2], [0.1, 0.9]]
[[ 0.8  1. ]
 [ 0.1  1. ]]


In [44]:
tm = [[0.9,0.1],[0.1,0.9]]
prior = ((-1., 1.0, 0.5),(2.3, 1.0, 0.5)) # mean, var, belive
bs = BootstrapFilter(prior, tm)

(5000, 3)


/home/arne/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:15: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/arne/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:16: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/arne/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:22: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/arne/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
